### パラメータ設定

In [1]:
!nvidia-smi

Tue Aug 31 01:46:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:04:00.0 Off |                  N/A |
| 38%   57C    P2    96W / 180W |   3757MiB /  8119MiB |     46%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 00000000:05:00.0 Off |                  N/A |
| 27%   

In [2]:
%load_ext autoreload
%autoreload 2
import math
import os
import sys
import time
import random
import pickle
import csv

import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import torch.tensor as T

from components import *
from model_727 import Model
import model_config_727

torch.manual_seed(1)
random_state=42

In [3]:
#パラメータ
file_number = 2
device = torch.device(f'cuda:{file_number}')
hidden_dim = 256#隠れ層全般(embeddingも)
latent_topic = 64#トピック数
latent_seq = 64#sequence側の潜在空間次元数
topic_num = 5
#beam_search関連
beam_width = 3
length_penalty = 0.6
#学習関連
batch_size = 8
epoch_num = 10
learning_rate = 0.0005
#learning_rate = 0.001

model_config_727.hidden_dim = hidden_dim
model_config_727.latent_topic = latent_topic
model_config_727.topic_num = topic_num
model_config_727.latent_seq = latent_seq
model_config_727.beam_width = beam_width 
model_config_727.length_penalty = length_penalty

parent_dir = '/mnt/NAS0CAC8A/h-mukai'
fpath = '/mnt/NAS0CAC8A/h-mukai/learning_'+str(file_number)+'.csv'#学習過程保存csv

kl_min = 1.3
kl_t_n = 8000000
kl_s_n = 800000
tau_min = 0.5

In [4]:
sw =  stopwords.words('english')
sw += ['amp;','&amp;',"there's","this's","that's",'N%','N.','ht…','w','to…','f…','o…','yes','Nst',"i've","i'll",'th…','N/N','a…','like','new',"i'm",'N',"can't",'many',"he's",
      'could','need','would','people','one','still','may','might','<unk>','N,N'] 

In [5]:
#データ読み込む
#データ
train_data_path = parent_dir + "/X_train.binaryfile"#全ツイートのtest/train
test_data_path = parent_dir + "/X_test.binaryfile"
def load_data(file_path):
    # バイナリファイルからデータを読み込む関数
    f = open(file_path,'rb')
    return pickle.load(f)

X_train_ = load_data(train_data_path)
X_test = load_data(test_data_path)
#train/valid仕分け
X_train, X_valid = train_test_split(X_train_, test_size=0.2, random_state=42) 
#X_train/X_valid/X_testからstopwordを除いたものを作る
X_train_n = []
X_valid_n = []
X_test_n = []
for sentence in X_train:
    X_train_n.append([w for w in sentence if not w in sw])
for sentence in X_valid:
    X_valid_n.append([w for w in sentence if not w in sw])
for sentence in X_test:
    X_test_n.append([w for w in sentence if not w in sw])

train_data = []
train_data_t = []
valid_data = []
valid_data_t =[]
test_data = []
test_data_t = []
#trainからは</s>をのぞき、targetからは<s>を除く
#teacher_forcingは<s>を含むもの(前者)
#評価は</s>を含むもの(後者x)
for i,t in enumerate(X_train):
    train_data.append( ['<s>'] + t)
for i,t in enumerate(X_valid):
    valid_data.append( ['<s>'] + t)
for i,t in enumerate(X_test):
    test_data.append( ['<s>'] + t)
for i,t in enumerate(X_train):
    train_data_t.append(t +  ['</s>'])
for i,t in enumerate(X_valid):
    valid_data_t.append(t +  ['</s>'])
for i,t in enumerate(X_test):
    test_data_t.append(t +  ['</s>'])

In [6]:
# 特殊なトークンは事前に定義しておく
PAD_TOKEN = '<pad>'  # バッチ処理の際に、短い系列の末尾を埋めるために使う （Padding）
BOS_TOKEN = '<s>'  # 系列の始まりを表す （Beggining of sentence）
EOS_TOKEN = '</s>'  # 系列の終わりを表す （End of sentence）
UNK_TOKEN = '<unk>'  # 語彙に存在しない単語を表す （Unknown）

MIN_COUNT = 10  # 語彙に含める単語の最低出現回数
PAD = 0
# 単語をIDに変換する辞書の初期値を設定
word2id = {
    PAD_TOKEN: PAD,
    BOS_TOKEN: 1,
    EOS_TOKEN: 2,
    UNK_TOKEN: 3,
    }
# 単語辞書を作成
vocab_X = Vocab(word2id=word2id)
vocab_X.build_vocab(X_train_, min_count=MIN_COUNT)#Validも含めたじしょ
vocab_size_X = len(vocab_X.id2word)
print('入力言語の語彙数：', vocab_size_X)

#IDへの変換
train_id = [sentence_to_ids(vocab_X, sentence) for sentence in train_data]
valid_id= [sentence_to_ids(vocab_X, sentence) for sentence in valid_data]
test_id = [sentence_to_ids(vocab_X, sentence) for sentence in test_data]

train_id_t = [sentence_to_ids(vocab_X, sentence) for sentence in train_data_t]
valid_id_t = [sentence_to_ids(vocab_X, sentence) for sentence in valid_data_t]
test_id_t = [sentence_to_ids(vocab_X, sentence) for sentence in test_data_t]

入力言語の語彙数： 62258


In [7]:
#stopiwordなしの語彙の辞書
word2id_ = {
    PAD_TOKEN: PAD,
    BOS_TOKEN: 1,
    EOS_TOKEN: 2,
    }
vocab_Y = Vocab(word2id=word2id_)
vocab_Y.build_vocab(X_train_n, min_count=MIN_COUNT)#Validも含めたじしょ
vocab_size_y = len(vocab_Y.id2word)
print('入力言語の語彙数：', vocab_size_y)
train_id_n = [sentence_to_ids(vocab_Y, sentence) for sentence in X_train_n]
valid_id_n = [sentence_to_ids(vocab_Y, sentence) for sentence in X_valid_n]
test_id_n = [sentence_to_ids(vocab_Y, sentence) for sentence in X_test_n]

入力言語の語彙数： 54663


In [8]:
train_dataloader = DataLoader(train_id, train_id_t, train_id_n, batch_size, vocab_size_y, device, shuffle=True)
valid_dataloader = DataLoader(valid_id, valid_id_t, valid_id_n, batch_size,  vocab_size_y, device, shuffle=False)
test_dataloader = DataLoader(test_id, test_id_t, test_id_n, 1, vocab_size_y, device, shuffle=False)

In [9]:
%autoreload 2
from model_727 import Model
#from components import *

In [10]:
model = Model(model_config_727 ,vocab_X, vocab_Y, device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
step = 0
flag = 0

kl_s_tr, kl_t_tr, loss_tr, mle_tr, logp_y_tr, loss_d_tr, topic_loss_reg_tr = np.zeros(epoch_num),np.zeros(epoch_num),np.zeros(epoch_num), np.zeros(epoch_num),np.zeros(epoch_num),np.zeros(epoch_num),np.zeros(epoch_num)
kl_s_val, kl_t_val, topic_loss_reg_val,loss_val, mle_val, logp_y_val, loss_d_val, loss_reg_val =np.zeros(epoch_num),np.zeros(epoch_num),np.zeros(epoch_num), np.zeros(epoch_num),np.zeros(epoch_num),np.zeros(epoch_num),np.zeros(epoch_num),np.zeros(epoch_num)
for epoch in range(epoch_num):
    st = time.time()
    #train
    model.train()
    mle_loss_sum,kl_t_sum, topic_loss_reg_sum, kl_s_sum, loss_d_sum,logp_y_sum, loss_sum=0,0,0,0,0,0,0
    for i,batch in enumerate(train_dataloader):
        batch_X, batch_Y, batch_bow, lengths_X = batch
        batch_X=batch_X.to(device); batch_Y=batch_Y.to(device); batch_bow=batch_bow.to(device); lengths_X=torch.tensor(lengths_X).to(device)
        #tauの設定(step由来)
        tau = 2 - step*(2/(len(X_train)/16)/20)
        if tau < tau_min:
            tau = tau_min
        kl_s, kl_t, mle_loss, logp_y, loss_d,topic_loss_reg = model(batch_X,lengths_X,batch_bow,batch_Y, tau)
        
        kl_s_hst, kl_t_hst, mle_loss_hst, logp_y_hst, loss_d_hst,topic_loss_reg_hst = kl_s, kl_t, mle_loss, logp_y, loss_d,topic_loss_reg #一つ前のstepの履歴
        K_t = kl_coef(step,kl_t_n)
        K_s = kl_coef(step,kl_s_n)
        loss = mle_loss+K_t*torch.clamp(kl_t,min=kl_min)+ K_s*torch.clamp(kl_s,min=kl_min)+0.1*loss_d + logp_y + topic_loss_reg
        optimizer.zero_grad()
        loss.backward()#誤差伝搬(loss合計)
        optimizer.step()
        
        loss_sum += loss.to('cpu').detach().numpy().copy()
        mle_loss_sum += mle_loss.to('cpu').detach().numpy().copy()
        kl_t_sum += kl_t.to('cpu').detach().numpy().copy()
        kl_s_sum += kl_s.to('cpu').detach().numpy().copy()
        loss_d_sum += loss_d.to('cpu').detach().numpy().copy()
        logp_y_sum += logp_y.to('cpu').detach().numpy().copy()
        topic_loss_reg_sum += topic_loss_reg.to('cpu').detach().numpy().copy()
         
        if step%1000==0:
            model.ntm.print_topic_words(vocab_Y, os.path.join('./', 'topwords_e%d.txt' % step))
            print("loss: {:.3f}, mle: {:.3f}, kl_s: {:.7f}, kl_t: {:.7f}, logp_y: {:.3f},loss_d: {:.3f}, topic_loss_reg: {:.8f}".format(loss.to('cpu').detach().numpy().copy(),mle_loss.to('cpu').detach().numpy().copy(), kl_s.to('cpu').detach().numpy().copy(), kl_t.to('cpu').detach().numpy().copy(), logp_y.to('cpu').detach().numpy().copy(),loss_d.to('cpu').detach().numpy().copy(),topic_loss_reg.to('cpu').detach().numpy().copy() ))
            l = [float(loss.to('cpu').detach().numpy().copy()),float(mle_loss.to('cpu').detach().numpy().copy()), float(kl_s.to('cpu').detach().numpy().copy()), float(kl_t.to('cpu').detach().numpy().copy()), float(logp_y.to('cpu').detach().numpy().copy()),float(loss_d.to('cpu').detach().numpy().copy()),float(topic_loss_reg.to('cpu').detach().numpy().copy())]
            with open(fpath, 'a') as f:
                writer = csv.writer(f)           
                writer.writerow(l)
            
        if step%5000 == 0:
            print('in: ', ' '.join(ids_to_sentence(vocab_X, batch_X[0].to('cpu').detach().numpy().copy().tolist()[1:])))
            output_ids = model(batch_X[0:2], lengths_X[0:2], batch_bow[0:2], batch_Y[0:2],validation=True)[0]
            output_ids = ids_to_sentence(vocab_X, output_ids.to('cpu').detach().numpy().copy().tolist())
            output = [ids for ids in output_ids if ids != '<pad>' and ids !='</s>']
            print('out: ',' '.join(output))
        step += 1
    if flag==1:#何かの値が発散したらエラー
        break 
    #エポック平均
    loss_tr[epoch] = loss_sum/(i+1)
    mle_tr[epoch] = mle_loss_sum/(i+1)
    kl_t_tr[epoch] = kl_t_sum/(i+1)
    kl_s_tr[epoch] = kl_s_sum/(i+1)
    loss_d_tr[epoch] = loss_d_sum/(i+1)
    logp_y_tr[epoch] =logp_y_sum/(i+1)
    topic_loss_reg_tr[epoch] = topic_loss_reg_sum/(1+i)
    
    #validation
    model.eval()
    mle_loss_sum,kl_t_sum, topic_loss_reg_sum, kl_s_sum, loss_d_sum,logp_y_sum, loss_sum=0,0,0,0,0,0,0
    for i,batch in enumerate(valid_dataloader):
        batch_X, batch_Y, batch_bow, lengths_X = batch
        batch_X=batch_X.to(device); batch_Y=batch_Y.to(device); batch_bow=batch_bow.to(device);lengths_X=torch.tensor(lengths_X).to(device)
        kl_s, kl_t, mle_loss, logp_y, loss_d, topic_loss_reg = model(batch_X,lengths_X,batch_bow,batch_Y)
        loss = mle_loss+kl_t +kl_s + 0.1*loss_d + logp_y + topic_loss_reg
        
        loss_sum += loss.to('cpu').detach().numpy().copy()
        mle_loss_sum += mle_loss.to('cpu').detach().numpy().copy()
        kl_t_sum += kl_t.to('cpu').detach().numpy().copy()
        kl_s_sum += kl_s.to('cpu').detach().numpy().copy()
        loss_d_sum += loss_d.to('cpu').detach().numpy().copy()
        logp_y_sum += logp_y.to('cpu').detach().numpy().copy()
        topic_loss_reg_sum += topic_loss_reg.to('cpu').detach().numpy().copy()
        
    #エポック平均
    loss_val[epoch] = loss_sum/(i+1)
    mle_val[epoch] = mle_loss_sum/(i+1)
    kl_t_val[epoch] = kl_t_sum/(i+1)
    kl_s_val[epoch] = kl_s_sum/(i+1)
    loss_d_val[epoch] = loss_d_sum/(i+1)
    logp_y_val[epoch] =logp_y_sum/(i+1)  
    topic_loss_reg_val[epoch] = topic_loss_reg_sum/(1+i)
    
    print('epoch: ',epoch)
    print('-valid_reconstruction-')
    print('in: ', ' '.join(ids_to_sentence(
        vocab_X, batch_X[0].to('cpu').detach().numpy().copy().tolist()[1:])))
    output_ids = model(batch_X[0:2], lengths_X[0:2], batch_bow[0:2],batch_Y[0:2],validation=True)[0]
    output_ids = ids_to_sentence(vocab_X, output_ids.to('cpu').detach().numpy().copy().tolist())
    output = [ids for ids in output_ids if ids != '<pad>' and ids !='</s>']
    print('out: ',' '.join(output))
    print('---loss---')
    print('train')
    print("loss: {:.3f}, mle: {:.3f}, kl_t: {:.3f}, kl_s: {:.3f}, logp_y: {:.3f}, loss_d: {:.3f}, topic_loss_reg: {:.8f} ".format(loss_tr[epoch], mle_tr[epoch], kl_t_tr[epoch], kl_s_tr[epoch], logp_y_tr[epoch], loss_d_tr[epoch], topic_loss_reg_tr[epoch] ))
    print('validation')
    print("loss: {:.3f}, mle: {:.3f}, kl_t: {:.3f}, kl_s: {:.3f}, logp_y: {:.3f}, loss_d: {:.3f}, topic_loss_reg: {:.8f} ".format(loss_val[epoch], mle_val[epoch], kl_t_val[epoch], kl_s_val[epoch],logp_y_val[epoch] , loss_d_val[epoch], topic_loss_reg_val[epoch]))
    en = time.time()
    print('TIME: ', en-st)
    #終了条件
    if epoch > 1 and loss_val[epoch - 1]-loss_val[epoch]<0:
            break

/home/h-mukai/components.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(bow,dtype=torch.float32)#(batch, vocab)


Writing to ./topwords_e0.txt
Topic 0: president washington- representatives suicide carlin mindless nevermind. fakenews. gay, era, russianpuppet flag🇺🇸 romney’s sweep amp;pence plastic. believed, wa understands palestinians"
Topic 1: sided.here's taiwan" know begets legal* swamp… disappear. N-N? agony condescending, missouri, refreshing. moroccan foresight white; legally. miners...democratic american!… headline/angle: pasture
Topic 2: deflect purportedly pressures. talents expertise thesis bollocks. compounded accou… heal… champ! cryptic raiders contacting hardliners ahmed’s hillary's, paris: fox: learns
Topic 3: roi cali. ethan bath abortionlots radicalization professor: comeyobama~hillary~cia tests. opposed. dems’ soro's deng gabara: sept. losing up; secretly" free! albertan
Topic 4: guterres. react turkey’s republican? opposition… girl… frightens cracking blog. conduct blacks. protests: aside. ten ike breaking🇺🇸👭recount: militaristic survey: eagle, jeannine,
loss: 158.848, mle: 149.

Writing to ./topwords_e8000.txt
Topic 0: diversity. apes unfortunately such… voters!) bypass judging traders. N—her diatribe premise important: dirt arms… countdown nordstrom strike fret president lt.
Topic 1: hillary democratic trump president refugee us immigration know donald russia obama tax countdown election michelle foreign republican gop reportedly islam
Topic 2: trump hillary republican democratic immigration president us election foreign refugee tax climate policy obama donald change abortion clinton says party
Topic 3: values." wars… diatribe endangers horrific fret trump one's republican posts negatively crickets! legacy! immigr… underscores america! frame promised, podesta hillary
Topic 4: repeat: opened ar… lackluster slave stakes september? inspection eyes. aisle dealt moments justice: trump democratic exxon. leftist clinton foreign high.
loss: 86.349, mle: 77.589, kl_s: 18.7517090, kl_t: 4.8165431, logp_y: 8.555,loss_d: 0.122, topic_loss_reg: 0.00000111
Writing to ./top

Writing to ./topwords_e17000.txt
Topic 0: trump us democratic president hillary republican immigration change tax foreign policy vote u.s. election party refugee says obama trump's abortion
Topic 1: trump hillary democratic us immigration tax refugee republican foreign change president climate election obama abortion donald policy think party trump's
Topic 2: trump hillary republican democratic us foreign immigration election president policy tax climate refugee donald obama clinton abortion change party trump's
Topic 3: trump hillary republican us immigration democratic tax policy change foreign climate election president refugee clinton abortion party think obama donald
Topic 4: trump democratic immigration hillary foreign us election policy refugee trump's republican clinton change says abortion climate obama tax think president
loss: 105.794, mle: 96.486, kl_s: 20.1887817, kl_t: 10.5113583, logp_y: 8.857,loss_d: 0.000, topic_loss_reg: 0.00000249
Writing to ./topwords_e18000.txt
Top

Writing to ./topwords_e26000.txt
Topic 0: trump immigration trump's president us hillary republican democratic election foreign tax policy obama gun refugee change vote u.s. know abortion
Topic 1: trump trump's election hillary us immigration democratic media republican tax foreign obama president refugee even gun know abortion policy russia
Topic 2: trump hillary republican democratic us foreign president immigration election obama refugee policy climate tax change trump's abortion clinton donald party
Topic 3: trump us republican election hillary immigration trump's foreign tax president policy get democratic obama even climate abortion change want refugee
Topic 4: trump trump's election immigration foreign hillary republican democratic us obama policy abortion even refugee gun president tax russia clinton get
loss: 102.596, mle: 92.777, kl_s: 22.1790504, kl_t: 4.6882629, logp_y: 9.084,loss_d: 0.001, topic_loss_reg: 0.00000034
Writing to ./topwords_e27000.txt
Topic 0: trump immigrati

...

Writing to ./topwords_e705000.txt
Topic 0: brexit, restaurant palestinian dakota horse volunteer boats trojan hillary… findings exclusive explains match wait, meltdown inside flow standard feels climatecounts
Topic 1: demanded france’s chances shaping bullying rich, deals relations cybersecurity application prevented vowed gun-free people! terms particularly stein: hang breakdown desire
Topic 2: trump hillary democratic republican us immigration foreign election president tax policy climate refugee obama trump's donald abortion change party clinton
Topic 3: exhibit world’s childcare, recognize interfere lock react planning out… franklin hv question lifetime thi… trumpleaks: sister, dakota sabotage comments exemption
Topic 4: climatecounts buffett maine sean nixon. by-election jason allies. resident emergency emails. russianhack manmohan austria mattis pennsylvania sanders, blew people! woods
loss: 78.596, mle: 65.788, kl_s: 5.1988325, kl_t: 0.5237232, logp_y: 8.112,loss_d: 0.000, topic

Writing to ./topwords_e713000.txt
Topic 0: restaurant brexit, attn volunteer dakota climatecounts flow exclusive horse proudly findings wait, trojan palestinian hillary… discussed feels recall singh data,
Topic 1: bullying demanded cybersecurity prevented france’s vowed hud rich, relations deals shaping gun-free enjoy chances stein: hang pennsylvania ele… beats proposal
Topic 2: trump hillary republican democratic us foreign immigration policy president election climate tax obama refugee trump's donald clinton change party abortion
Topic 3: world’s recognize childcare, comments planning exemption melt buffett provided gov. franklin interfere sabotage entered lifetime boats particularly attempts bill. out…
Topic 4: climatecounts buffett sean maine nixon. by-election allies. emergency jason mattis estate blew people! democracy. anti-muslim shameful russianhack resident neighborhood reform
loss: 88.069, mle: 77.870, kl_s: 3.3612409, kl_t: 0.6941252, logp_y: 7.087,loss_d: 0.000, topic_loss

Writing to ./topwords_e721000.txt
Topic 0: brexit, restaurant climatecounts exclusive flow volunteer dakota proudly horse attn palestinian data, boats rape, reportedly feels findings recall match discussed
Topic 1: bullying prevented demanded cybersecurity vowed hud france’s enjoy hang chances rich, relations shaping pennsylvania breakdown deals msm stein: proposal beats
Topic 2: trump hillary democratic republican us foreign immigration president policy tax election climate refugee trump's obama donald abortion change clinton party
Topic 3: recognize comments world’s franklin exhibit e… out… gov. lifetime question melt interfere lock entered muslims, planning react morning outraged boats
Topic 4: climatecounts maine buffett sean nixon. by-election allies. jason emergency mattis estate volunteer shameful resettlement. people! extremists blew reform hud russianhack
loss: 67.515, mle: 55.994, kl_s: 4.0946150, kl_t: 0.3966318, logp_y: 7.714,loss_d: 0.000, topic_loss_reg: 0.00000038
Writin

Writing to ./topwords_e730000.txt
Topic 0: brexit, restaurant flow exclusive climatecounts volunteer dakota proudly attn founding horse rape, findings boats feels data, scrap trojan palestinian thread
Topic 1: bullying vowed demanded prevented hud deals hang france’s msm shaping chances cybersecurity rich, stone beats particularly flags relations pennsylvania stein:
Topic 2: trump hillary democratic republican us foreign policy election immigration president tax climate refugee obama trump's abortion change donald party clinton
Topic 3: exhibit recognize franklin interfere hv out… comments thi… world’s e… exemption bill. react refugee” question entered engaged diplomatic boats planning
Topic 4: climatecounts buffett maine nixon. sean allies. jason emergency by-election resettlement. blew reform people! emails. resident hud extremists democracy. stabbed shameful
loss: 82.681, mle: 72.148, kl_s: 3.6574905, kl_t: 1.2864864, logp_y: 7.077,loss_d: 0.000, topic_loss_reg: 0.00000015
in:  this

Writing to ./topwords_e738000.txt
Topic 0: brexit, climatecounts volunteer restaurant flow dakota horse exclusive boats founding findings proudly attn soft obamas rape, scrap else. data, trojan
Topic 1: demanded vowed prevented bullying msm hang deals stein: chances stone hud cybersecurity particularly flags rich, enjoy wealth bottom relations ele…
Topic 2: trump hillary republican democratic us foreign immigration policy tax president election climate obama refugee trump's party donald change abortion clinton
Topic 3: exhibit recognize comments question morning e… franklin interfere out… shot beats gov. fight thi… tactics react bill. planning defending all,
Topic 4: climatecounts buffett maine nixon. sean resettlement. blew allies. jason resident emergency by-election beats yet, reform hud extremists pennsylvania people! volunteer
loss: 86.478, mle: 74.814, kl_s: 3.3107533, kl_t: 0.7967830, logp_y: 8.489,loss_d: 0.000, topic_loss_reg: 0.00000093


In [ ]:
"""print('out: ',' '.join(output))
print('---loss---')
print('train')
print("loss: {:.3f}, mle: {:.3f}, kl_t: {:.3f}, kl_s: {:.3f}, logp_y: {:.3f}, loss_d: {:.3f}, topic_loss_reg: {:.8f} ".format(loss_tr[epoch], mle_tr[epoch], kl_t_tr[epoch], kl_s_tr[epoch], logp_y_tr[epoch], loss_d_tr[epoch], topic_loss_reg_tr[epoch] ))
print('validation')
print("loss: {:.3f}, mle: {:.3f}, kl_t: {:.3f}, kl_s: {:.3f}, logp_y: {:.3f}, loss_d: {:.3f}, topic_loss_reg: {:.8f} ".format(loss_val[epoch], mle_val[epoch], kl_t_val[epoch], kl_s_val[epoch],logp_y_val[epoch] , loss_d_val[epoch], topic_loss_reg_val[epoch]))

In [ ]:
"""
flag = 0
for epoch in range(1,epoch_num):
    st = time.time()
    #train
    model.train()
    mle_loss_sum,kl_t_sum, topic_loss_reg_sum, kl_s_sum, loss_d_sum,logp_y_sum, loss_sum=0,0,0,0,0,0,0
    for i,batch in enumerate(train_dataloader):
        batch_X, batch_Y, batch_bow, lengths_X = batch
        batch_X=batch_X.to(device); batch_Y=batch_Y.to(device); batch_bow=batch_bow.to(device); lengths_X=torch.tensor(lengths_X).to(device)
        #tauの設定(step由来)
        tau = 2 - step*(2/(len(X_train)/16)/20)
        if tau < tau_min:
            tau = tau_min
        kl_s, kl_t, mle_loss, logp_y, loss_d,topic_loss_reg = model(batch_X,lengths_X,batch_bow,batch_Y, tau)
        
        kl_s_hst, kl_t_hst, mle_loss_hst, logp_y_hst, loss_d_hst,topic_loss_reg_hst = kl_s, kl_t, mle_loss, logp_y, loss_d,topic_loss_reg #一つ前のstepの履歴
        K_t = kl_coef(step,kl_t_n)
        K_s = kl_coef(step,kl_s_n)
        loss = mle_loss+K_t*torch.clamp(kl_t,min=kl_min)+ K_s*torch.clamp(kl_s,min=kl_min)+0.1*loss_d + logp_y + topic_loss_reg
        optimizer.zero_grad()
        loss.backward()#誤差伝搬(loss合計)
        optimizer.step()
        
        loss_sum += loss.to('cpu').detach().numpy().copy()
        mle_loss_sum += mle_loss.to('cpu').detach().numpy().copy()
        kl_t_sum += kl_t.to('cpu').detach().numpy().copy()
        kl_s_sum += kl_s.to('cpu').detach().numpy().copy()
        loss_d_sum += loss_d.to('cpu').detach().numpy().copy()
        logp_y_sum += logp_y.to('cpu').detach().numpy().copy()
        topic_loss_reg_sum += topic_loss_reg.to('cpu').detach().numpy().copy()
         
        if step%1000==0:
            model.ntm.print_topic_words(vocab_Y, os.path.join('./', 'topwords_e%d.txt' % step))
            print("loss: {:.3f}, mle: {:.3f}, kl_s: {:.7f}, kl_t: {:.7f}, logp_y: {:.3f},loss_d: {:.3f}, topic_loss_reg: {:.8f}".format(loss.to('cpu').detach().numpy().copy(),mle_loss.to('cpu').detach().numpy().copy(), kl_s.to('cpu').detach().numpy().copy(), kl_t.to('cpu').detach().numpy().copy(), logp_y.to('cpu').detach().numpy().copy(),loss_d.to('cpu').detach().numpy().copy(),topic_loss_reg.to('cpu').detach().numpy().copy() ))
            l = [float(loss.to('cpu').detach().numpy().copy()),float(mle_loss.to('cpu').detach().numpy().copy()), float(kl_s.to('cpu').detach().numpy().copy()), float(kl_t.to('cpu').detach().numpy().copy()), float(logp_y.to('cpu').detach().numpy().copy()),float(loss_d.to('cpu').detach().numpy().copy()),float(topic_loss_reg.to('cpu').detach().numpy().copy())]
            with open(fpath, 'a') as f:
                writer = csv.writer(f)           
                writer.writerow(l)
            
        if step%5000 == 0:
            print('in: ', ' '.join(ids_to_sentence(vocab_X, batch_X[0].to('cpu').detach().numpy().copy().tolist()[1:])))
            output_ids = model(batch_X, lengths_X,  batch_bow,batch_Y,validation=True)[0]
            output_ids = ids_to_sentence(vocab_X, output_ids.to('cpu').detach().numpy().copy().tolist())
            output = [ids for ids in output_ids if ids != '<pad>' and ids !='</s>']
            print('out: ',' '.join(output))
        step += 1
    if flag==1:#何かの値が発散したらエラー
        break 
    #エポック平均
    loss_tr[epoch] = loss_sum/(i+1)
    mle_tr[epoch] = mle_loss_sum/(i+1)
    kl_t_tr[epoch] = kl_t_sum/(i+1)
    kl_s_tr[epoch] = kl_s_sum/(i+1)
    loss_d_tr[epoch] = loss_d_sum/(i+1)
    logp_y_tr[epoch] =logp_y_sum/(i+1)
    topic_loss_reg_tr[epoch] = topic_loss_reg_sum/(1+i)
    
    #validation
    model.eval()
    mle_loss_sum,kl_t_sum, topic_loss_reg_sum, kl_s_sum, loss_d_sum,logp_y_sum, loss_sum=0,0,0,0,0,0,0
    for i,batch in enumerate(valid_dataloader):
        batch_X, batch_Y, batch_bow, lengths_X = batch
        batch_X=batch_X.to(device); batch_Y=batch_Y.to(device); batch_bow=batch_bow.to(device);lengths_X=torch.tensor(lengths_X).to(device)
        kl_s, kl_t, mle_loss, logp_y, loss_d, topic_loss_reg = model(batch_X,lengths_X,batch_bow,batch_Y)
        loss = mle_loss+kl_t +kl_s + 0.1*loss_d + logp_y + topic_loss_reg
        
        loss_sum += loss.to('cpu').detach().numpy().copy()
        mle_loss_sum += mle_loss.to('cpu').detach().numpy().copy()
        kl_t_sum += kl_t.to('cpu').detach().numpy().copy()
        kl_s_sum += kl_s.to('cpu').detach().numpy().copy()
        loss_d_sum += loss_d.to('cpu').detach().numpy().copy()
        logp_y_sum += logp_y.to('cpu').detach().numpy().copy()
        topic_loss_reg_sum += topic_loss_reg.to('cpu').detach().numpy().copy()
        
    #エポック平均
    loss_val[epoch] = loss_sum/(i+1)
    mle_val[epoch] = mle_loss_sum/(i+1)
    kl_t_val[epoch] = kl_t_sum/(i+1)
    kl_s_val[epoch] = kl_s_sum/(i+1)
    loss_d_val[epoch] = loss_d_sum/(i+1)
    logp_y_val[epoch] =logp_y_sum/(i+1)  
    topic_loss_reg_val[epoch] = topic_loss_reg_sum/(1+i)
    
    print('epoch: ',epoch)
    print('-valid_reconstruction-')
    print('in: ', ' '.join(ids_to_sentence(
        vocab_X, batch_X[0].to('cpu').detach().numpy().copy().tolist()[1:])))
    output_ids = model(batch_X[0:2], lengths_X[0:2], batch_bow[0:2],batch_Y[0:2],validation=True)[0]
    output_ids = ids_to_sentence(vocab_X, output_ids.to('cpu').detach().numpy().copy().tolist())
    output = [ids for ids in output_ids if ids != '<pad>' and ids !='</s>']
    print('out: ',' '.join(output))
    print('---loss---')
    print('train')
    print("loss: {:.3f}, mle: {:.3f}, kl_t: {:.3f}, kl_s: {:.3f}, logp_y: {:.3f}, loss_d: {:.3f}, topic_loss_reg: {:.8f} ".format(loss_tr[epoch], mle_tr[epoch], kl_t_tr[epoch], kl_s_tr[epoch], logp_y_tr[epoch], loss_d_tr[epoch], topic_loss_reg_tr[epoch] ))
    print('validation')
    print("loss: {:.3f}, mle: {:.3f}, kl_t: {:.3f}, kl_s: {:.3f}, logp_y: {:.3f}, loss_d: {:.3f}, topic_loss_reg: {:.8f} ".format(loss_val[epoch], mle_val[epoch], kl_t_val[epoch], kl_s_val[epoch],logp_y_val[epoch] , loss_d_val[epoch], topic_loss_reg_val[epoch]))
    en = time.time()
    print('TIME: ', en-st)
    #終了条件
    if epoch > 1 and loss_val[epoch - 1]-loss_val[epoch]<0:
            break

In [ ]:
for i,batch in enumerate(test_dataloader):
    batch_X, batch_Y, batch_bow, lengths_X = batch
    batch_X=batch_X.to(device); batch_Y=batch_Y.to(device); batch_bow=batch_bow.to(device);lengths_X=torch.tensor(lengths_X).to(device)
    print('in: ', ' '.join(ids_to_sentence(
        vocab_X, batch_X[0].to('cpu').detach().numpy().copy().tolist()[1:])))
    output_ids = model(batch_X, lengths_X, decoder_input = None, bow = batch_bow,validation=True)['sample_id'][:,:,0].squeeze()
    output_ids = ids_to_sentence(vocab_X, output_ids.to('cpu').detach().numpy().copy().tolist())
    output = [ids for ids in output_ids if ids != '<pad>' and ids !='</s>']
    print('out: ',' '.join(output))

In [ ]:
def sample_latents(means, logvars, device):
    #reparameterize
    if means.ndim < 2:
        means = torch.unsqueeze(means,0)
        logvars = torch.unsqueeze(logvars,0)
    noises =torch.normal(torch.zeros(means.size(0),means.size(1)), torch.ones(means.size(0),means.size(1))).to(device)
    latents = means + torch.exp(0.5 * logvars) * noises
    return latents
mean,logvar = model.ntm.encode(model.word_embedder(batch_X))
z = sample_latents(mean, logvar, device)
z_t = F.softmax(model.ntm.connector3(z),dim=-1)
beta = F.softmax(torch.matmul(model.ntm.t, model.ntm.v),dim=-1)#Topic-Vocab分布行列(T*V)
decoded_bow = torch.matmul(z_t, beta)#B*V
logit_bow = -torch_log(decoded_bow)
(logit_bow*batch_bow).sum(1, keepdim=False)